In [73]:
import os
import hjson
import string


In [74]:

def is_ascii(s):
    return all(c in string.printable for c in s)

def extract_ascii_displaynames(input_folder, filter_string, output_folder):
    if os.path.abspath(input_folder) == os.path.abspath(output_folder):
        raise ValueError("Input folder and output folder cannot be the same.")
    
    os.makedirs(output_folder, exist_ok=True)
    # Clear the output folder
    for file in os.listdir(output_folder):
        file_path = os.path.join(output_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}: {e}")

    for filename in os.listdir(input_folder):
        result = {}
        if filename.endswith(".hjson") and filter_string in filename:
            file_path = os.path.join(input_folder, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = hjson.load(f)
                except Exception as e:
                    print(f"Failed to parse {filename}: {e}")
                    continue

                for key, value in data.items():
                    try:
                        display_name = value.get("DisplayName", "")
                        if isinstance(display_name, str) and display_name != "" and is_ascii(display_name) and not '{' in display_name and not '}' in display_name and not all(c.isupper() for c in display_name if c.isalpha()):
                            result[key] = value
                    except Exception as e:
                        continue
        if len(result) > 0:
        # 输出为 HJSON 文件
            output_path = os.path.join(output_folder, filename)
            with open(output_path, 'w', encoding='utf-8') as f:
                hjson.dump(result, f, ensure_ascii=False, indent=4)
        


In [75]:
if __name__ == "__main__":
    input_folder = "../Localization/zh-Hans"
    filter_string = "Items"
    output_folder = "Output/Untranslated"
    extract_ascii_displaynames(input_folder, filter_string, output_folder)